<a href="https://colab.research.google.com/github/bhattacharjee/mtu-nlp-assignment/blob/main/web-examples/bert_qa_rahul_agarwal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch -q -q -q
!pip install transformers -q -q -q


     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 63.0 MB/s 
     |████████████████████████████████| 596 kB 70.9 MB/s 
     |████████████████████████████████| 61 kB 652 kB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 


In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
#from datasets import load_dataset, load_metric
import random
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Use cuda by default
cuda = torch.device('cuda')
torch.set_default_tensor_type('torch.cuda.FloatTensor')



# Use the default BERT pretrained on SQUAD for examples

In [6]:
import time

bert_model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]
for question in questions:
    t1 = time.monotonic()
    inputs = tokenizer.encode_plus(\
                question,\
                text,\
                add_special_tokens=True,\
                return_tensors="pt")

    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    pred = model(**inputs)
    answer_start_scores = pred['start_logits'][0]
    answer_end_scores = pred['end_logits'][0]

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    #print(answer_start_scores.detach().cpu().numpy().shape)
    #print(answer_end_scores.detach().cpu().numpy().shape)
    #print(answer_start, answer_end)

    #print("--->\n", inputs['input_ids'].detach().cpu().numpy())


    answer = tokenizer\
                    .convert_tokens_to_string(\
                        tokenizer.convert_ids_to_tokens(\
                            input_ids[answer_start:answer_end]))

    print(f"Time taken = {time.monotonic() - t1}")
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Time taken = 0.02152902400007406
Question: How many pretrained models are available in Transformers?
Answer: over 32 +

Time taken = 0.0221108450000429
Question: What does Transformers provide?
Answer: general - purpose architectures

Time taken = 0.02412749400014036
Question: Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch

